# Exploratory

## Notes

* **FGTS:** Refere-se a débitos de empregadores com o fundo de garantia.
* **PREV (Previdenciário):** Débitos de contribuições previdenciárias.
* **SIDA:** Conjunto de dados do sistema unificado da administração federal, com dados detalhados sobre devedores e sua situação perante a União.

Layout dos arquivos:

➜ FGTS PREV e SIDA
| Campo                     | Descrição                                                                 |
|---------------------------|---------------------------------------------------------------------------|
| CPF_CNPJ                  | CPF ou CNPJ do devedor inscrito na dívida ativa.                         |
| TIPO_PESSOA               | Indica se o devedor é pessoa física (F) ou jurídica (J).                 |
| TIPO_DEVEDOR              | Categoria do devedor: contribuinte individual, empresa, órgão público etc. |
| NOME_DEVEDOR              | Nome ou razão social do devedor.                                         |
| UF_DEVEDOR                | Unidade da Federação (estado) do devedor.                               |
| UNIDADE_RESPONSAVEL       | Unidade da PGFN responsável pela inscrição da dívida.                   |
| NUMERO_INSCRICAO          | Número único da inscrição da dívida ativa.                              |
| TIPO_SITUACAO_INSCRICAO   | Tipo da situação da inscrição: ativa, suspensa, extinta etc.            |
| SITUACAO_INSCRICAO        | Descrição detalhada da situação atual da dívida.                        |
| TIPO_CREDITO              | Tipo de crédito tributário ou não tributário (ex: FGTS, Multas).        |
| DATA_INSCRICAO            | Data da inscrição na dívida ativa.                                      |
| INDICADOR_AJUIZADO        | Indica se a dívida foi ajuizada: 'S' ou 'N'.                            |
| VALOR_CONSOLIDADO         | Valor total da dívida com acréscimos legais.                            |

➜ FGTS
| Campo                  | Descrição                                           |
|------------------------|---------------------------------------------------|
| ENTIDADE_RESPONSAVEL   | Entidade que gerou a inscrição do débito FGTS.    |
| UNIDADE_INSCRICAO      | Unidade/agência que realizou a inscrição do débito FGTS. |


## Imports

In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

import pandas as pd

from modules.utils import (
        find_zip_files_by_pattern,
        read_zip_csvs_as_df,
        dataframe_info,
        extract_origem_from_filename
    )

## Dados

In [2]:
zip_files = find_zip_files_by_pattern(directory=f"{project_root}/dados/", pattern='FGTS.zip')
all_dfs = []

for f in zip_files:
    print(f"Processando arquivo: {f}")
    df = read_zip_csvs_as_df(f, encoding='latin1', verbose=False)
    all_dfs.append(df)

final_df = pd.concat(all_dfs, ignore_index=True)

Processando arquivo: /home/nicolas/github/unb/ppca_mdm/dados/2020T2_Dados_abertos_FGTS.zip
Processando arquivo: /home/nicolas/github/unb/ppca_mdm/dados/2020T1_Dados_abertos_FGTS.zip


In [8]:
dataframe_info(final_df)


DataFrame Information:

Shape: (882198, 18)

Columns: ['CPF_CNPJ', 'TIPO_PESSOA', 'TIPO_DEVEDOR', 'NOME_DEVEDOR', 'UF_UNIDADE_RESPONSAVEL', 'UNIDADE_RESPONSAVEL', 'ENTIDADE_RESPONSAVEL', 'UNIDADE_INSCRICAO', 'NUMERO_INSCRICAO', 'TIPO_SITUACAO_INSCRICAO', 'SITUACAO_INSCRICAO', 'RECEITA_PRINCIPAL', 'DATA_INSCRICAO', 'INDICADOR_AJUIZADO', 'VALOR_CONSOLIDADO', '__source_file__', 'ANOMES', 'ORIGEM']

Data Types:
CPF_CNPJ                    object
TIPO_PESSOA                 object
TIPO_DEVEDOR                object
NOME_DEVEDOR                object
UF_UNIDADE_RESPONSAVEL      object
UNIDADE_RESPONSAVEL         object
ENTIDADE_RESPONSAVEL        object
UNIDADE_INSCRICAO           object
NUMERO_INSCRICAO            object
TIPO_SITUACAO_INSCRICAO     object
SITUACAO_INSCRICAO          object
RECEITA_PRINCIPAL           object
DATA_INSCRICAO              object
INDICADOR_AJUIZADO          object
VALOR_CONSOLIDADO          float64
__source_file__             object
ANOMES                      

In [3]:
#only numbers
final_df['CPF_CNPJ'] = (
    final_df['CPF_CNPJ']
    .astype(str)
    .str.replace(r'\D', '', regex=True)
    .where(final_df['CPF_CNPJ'].notna())
)

#extratcing year and month from filename
final_df["ANOMES"] = final_df["__source_file__"].str.extract(r'(\d{6})')


final_df["ORIGEM"] = final_df["__source_file__"].apply(extract_origem_from_filename)




In [ ]:
cnpj_counts = final_df['CPF_CNPJ'].value_counts()
duplicate_cnpjs = cnpj_counts[cnpj_counts > 1]
print(f"\nCPF_CNPJs duplicados:")
print(duplicate_cnpjs)

duplicate_rows = final_df[final_df['CPF_CNPJ'].isin(duplicate_cnpjs.index)]




CPF_CNPJs duplicados:
CPF_CNPJ
30477954000150    109
61575775000180    108
60477056000163     94
96653894000198     90
92219070000153     86
                 ... 
26925503000161      2
92635838000170      2
42407429000148      2
02795365000185      2
06266303000137      2
Name: count, Length: 241704, dtype: int64


In [5]:
duplicate_rows.loc[duplicate_rows['CPF_CNPJ'] == '26925503000161']

,CPF_CNPJ,TIPO_PESSOA,TIPO_DEVEDOR,NOME_DEVEDOR,UF_UNIDADE_RESPONSAVEL,UNIDADE_RESPONSAVEL,ENTIDADE_RESPONSAVEL,UNIDADE_INSCRICAO,NUMERO_INSCRICAO,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,DATA_INSCRICAO,INDICADOR_AJUIZADO,VALOR_CONSOLIDADO,__source_file__,ANOMES,ORIGEM
236634,26925503000161,Pessoa jurídica,Principal,J C IND E COMERCIO DE MATERIAIS ESPORTIVOS LTDA,NaN,Não se Aplica,CAIXA,GOIAS,FGGO199900108,Em cobrança,AJUIZADA,Contribuições FGTS,25/02/1999,SIM,10729.25,arquivo_lai_FGTS_NA_202006.csv,202006,FGTS
698236,26925503000161,Pessoa jurídica,Principal,J C IND E COMERCIO DE MATERIAIS ESPORTIVOS LTDA,NaN,Não se Aplica,CAIXA,GOIAS,FGGO199900108,Em cobrança,AJUIZADA,Contribuições FGTS,25/02/1999,SIM,10653.48,arquivo_lai_FGTS_NA_202002.csv,202002,FGTS


In [ ]:
count = final_df['NUMERO_INSCRICAO'].value_counts()
duplicates = count[count > 1]
print(duplicates)

duplicate_rows = final_df[final_df['NUMERO_INSCRICAO'].isin(duplicates.index)]


NUMERO_INSCRICAO
FGTO201400143    2
FGTO201500125    2
FGTO201600378    2
FGTO200800023    2
FGTO201900063    2
                ..
FGAC201000057    2
FGAC201000046    2
FGAC201700422    2
FGAC201100024    2
FGAC201500072    2
Name: count, Length: 436896, dtype: int64


In [ ]:
duplicate_rows.loc[duplicate_rows['NUMERO_INSCRICAO'] == 'FGTO201500125']

,CPF_CNPJ,TIPO_PESSOA,TIPO_DEVEDOR,NOME_DEVEDOR,UF_UNIDADE_RESPONSAVEL,UNIDADE_RESPONSAVEL,ENTIDADE_RESPONSAVEL,UNIDADE_INSCRICAO,NUMERO_INSCRICAO,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,DATA_INSCRICAO,INDICADOR_AJUIZADO,VALOR_CONSOLIDADO,__source_file__,ANOMES,ORIGEM
440886,10922861000180,Pessoa jurídica,Principal,TEIXEIRA & PEREIRA LTDA,TO,TOCANTINS,PGFN,TOCANTINS,FGTO201500125,Em cobrança,AJUIZADA,Contribuições FGTS,20/08/2015,SIM,8629.79,arquivo_lai_FGTS_TO_202006.csv,202006,FGTS
880810,10922861000180,Pessoa jurídica,Principal,TEIXEIRA & PEREIRA LTDA,TO,TOCANTINS,PGFN,TOCANTINS,FGTO201500125,Em cobrança,AJUIZADA,Contribuições FGTS,20/08/2015,SIM,8515.33,arquivo_lai_FGTS_TO_202002.csv,202002,FGTS


In [21]:
situacao_df = final_df[[
    'NUMERO_INSCRICAO',
    'ANOMES',
    'SITUACAO_INSCRICAO'
]].drop_duplicates().sort_values(by=['NUMERO_INSCRICAO', 'ANOMES'])

situacao_df['SITUACAO_INSCRICAO_ANTERIOR'] = situacao_df.groupby('NUMERO_INSCRICAO')['SITUACAO_INSCRICAO'].shift(1)

situacao_df['MUDOU_PARA_AJUIZADA'] = (
    (situacao_df['SITUACAO_INSCRICAO_ANTERIOR'] == 'INSCRITA') &
    (situacao_df['SITUACAO_INSCRICAO'] == 'AJUIZADA')
)

situacao_df = situacao_df[[
    'NUMERO_INSCRICAO',
    'MUDOU_PARA_AJUIZADA',
    'SITUACAO_INSCRICAO_ANTERIOR',
    'SITUACAO_INSCRICAO',
    'ANOMES'
]]



In [ ]:
situacao_df

NameError: name 'head' is not defined